<a href="https://colab.research.google.com/github/POS-Cross/Bravo-EDA/blob/main/ML/Bravo_Recommendation_based_on_frequently_bought_together_with_KMeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d khaledanaqwa/bravodata

! pip install mlxtend --upgrade


mkdir: cannot create directory ‘/root/.kaggle’: File exists
 97% 109M/112M [00:03<00:00, 25.6MB/s] 
100% 112M/112M [00:03<00:00, 34.9MB/s]


In [ ]:
import pandas as pd

CSV = '/content/bravodata.zip'

df_items=pd.read_csv(CSV)
df_items

,dTicketInternalKey,BravoNo,Invoice_Date,cBrutoValue,POS_NO,BillNo,dItemInternalKey,Barcode,ItemName,GrossRetailPrice,Quantity,TotalBeforeDisc,DiscValue,NetSales_IncVAT,Total_Net_InvoiceIncVAT,bHasPromotions,OfferNo,dVendorId,DeptNo,CashierNo,CustomerGroup,CustomerNo
0,30860300,1,01/09/2019 0:00:00,11.75,2,50200,142,121017,Tnuva Amek Cheese,48.0,0.24,11.75,1.95,9.80,18.8,-1,13369,103260.0,104,804,0,0
1,30860300,1,01/09/2019 0:00:00,6.00,2,50200,21718,252164,Healthy Bread Pcs,6.0,1.00,6.00,0.00,6.00,18.8,0,0,104540.0,107,804,0,0
2,30860300,1,01/09/2019 0:00:00,2.00,2,50200,75254,7622210622211,Cadbury 5Star 48G,2.0,1.00,2.00,0.00,2.00,18.8,0,0,106470.0,112,804,0,0
3,30860300,1,01/09/2019 0:00:00,1.00,2,50200,77885,6221007027751,Nesquik Biscuit Milk 56G,1.0,1.00,1.00,0.00,1.00,18.8,0,0,101520.0,112,804,0,0
4,30860301,1,01/09/2019 0:00:00,25.00,2,50201,1292,7622100919261,Marlboro Red,25.0,1.00,25.00,0.00,25.00,45.0,0,0,501024.0,108,804,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3481787,31755806,1,14/12/2019 0:00:00,30.00,5,38929,63798,8906021075107,Peanut Butter Ruparels Red 510G,15.0,2.00,30.00,10.00,20.00,268.9,-1,15967,100405.0,112,847,1,628959
3481788,31755806,1,14/12/2019 0:00:00,20.00,5,38929,68245,5449000127631,Jericho Water 8*1.5L Case,10.0,2.00,20.00,3.00,17.00,268.9,-1,16124,101730.0,112,847,1,628959
3481789,31755806,1,14/12/2019 0:00:00,22.00,5,38929,69643,7290001996553,Alaa Al Deen Tissues 800G 2P,22.0,1.00,22.00,2.00,20.00,268.9,-1,14084,101590.0,108,847,1,628959
3481790,31755806,1,14/12/2019 0:00:00,40.00,5,38929,76574,6253504290055,Reema Toilet 32 Roll,40.0,1.00,40.00,8.00,32.00,268.9,-1,15482,101590.0,108,847,1,628959


In [ ]:
# df_items = df_items.drop(columns=['Barcode','ItemName','Invoice_Date'])
df = df_items[['dTicketInternalKey','dItemInternalKey','bHasPromotions','OfferNo','DeptNo','CustomerNo']]
df = df.dropna()

df

,dTicketInternalKey,dItemInternalKey,bHasPromotions,OfferNo,DeptNo,CustomerNo
0,30860300,142,-1,13369,104,0
1,30860300,21718,0,0,107,0
2,30860300,75254,0,0,112,0
3,30860300,77885,0,0,112,0
4,30860301,1292,0,0,108,0
...,...,...,...,...,...,...
3481787,31755806,63798,-1,15967,112,628959
3481788,31755806,68245,-1,16124,112,628959
3481789,31755806,69643,-1,14084,108,628959
3481790,31755806,76574,-1,15482,108,628959


In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=6)
predict =km.fit_predict(df)


In [ ]:
df_items['predict']=predict

In [ ]:
groupbypredict=df_items.groupby('predict')

group0 = groupbypredict.get_group(0)
group1 = groupbypredict.get_group(1)
group2 = groupbypredict.get_group(2)
group3 = groupbypredict.get_group(3)
group4 = groupbypredict.get_group(4)
group5 = groupbypredict.get_group(5)


In [ ]:
products = df_items[['dItemInternalKey', 'ItemName']].copy()
products = products[~products.duplicated()]
products

,dItemInternalKey,ItemName
0,142,Tnuva Amek Cheese
1,21718,Healthy Bread Pcs
2,75254,Cadbury 5Star 48G
3,77885,Nesquik Biscuit Milk 56G
4,1292,Marlboro Red
...,...,...
3469260,19574,Cheese 600G Pasteurised
3469989,78303,Post Oreo Golden Cereal 311G
3471232,63789,Gullon Mini Cracker Biscuts 350G
3478939,56722,Cake 50 Prs


In [ ]:
# Set the index to StockCode
products = products.set_index('dItemInternalKey')

# Convert to Series for eve easier lookups
products = products['ItemName']
products

dItemInternalKey
142      Tnuva Amek Cheese                       
21718    Healthy Bread Pcs                       
75254    Cadbury 5Star 48G                       
77885    Nesquik Biscuit Milk 56G                
1292     Marlboro Red                            
                           ...                   
19574    Cheese 600G Pasteurised                 
78303    Post Oreo Golden Cereal 311G            
63789    Gullon Mini Cracker Biscuts 350G        
56722    Cake 50 Prs                             
53024    Karmel 11.99                            
Name: ItemName, Length: 9380, dtype: object

In [ ]:
products[12044]

'Safi Sunflower Oil 5L                   '

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

grouplist=[group0,group1,group2,group3,group4,group5]

for j in range(len(grouplist)):
  i = grouplist[j].dropna()

  i=i[['dTicketInternalKey','dItemInternalKey']]
  i = i.groupby('dTicketInternalKey')['dItemInternalKey'].apply(list).reset_index()
  te = TransactionEncoder()
  te.fit(i['dItemInternalKey'])
  orders_1hot = te.transform(i['dItemInternalKey'])
  grouplist[j] = pd.DataFrame(orders_1hot, columns =te.columns_)
  

In [ ]:
grouplist[4]

,1,71,74,76,77,78,79,80,81,127,128,139,140,141,142,145,146,148,149,251,255,256,257,281,294,296,297,298,302,303,304,305,306,307,308,310,312,313,321,326,...,78250,78251,78253,78254,78255,78258,78260,78261,78262,78264,78265,78266,78267,78268,78269,78272,78273,78274,78275,78276,78278,78279,78280,78281,78288,78289,78290,78291,78292,78293,78294,78301,78303,78308,78309,78310,78315,78317,78319,78323
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227835,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
227836,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules


%timeit -n1 -r1
aarrules=[]
for j in range(len(grouplist)):
  is_fp = fpgrowth(grouplist[j], min_support=0.00001, use_colnames=True)
  rules = association_rules(is_fp, metric="lift", min_threshold=10)
  aarrules.append(rules)


In [ ]:
def predict(antecedent, rules, max_results= 6):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a messy way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results].reset_index(drop=True)

In [ ]:
len(aarrules)

6

In [ ]:
itemset ={12044}
print(products[12044])
for j in range(len(aarrules)):
  # get the predictions
  preds = predict(itemset, aarrules[j])
  print(j,len(preds))
  # Display the descriptions of the predictions
  for stockid in preds:  
      print(j,products[stockid])

Safi Sunflower Oil 5L                   
0 0
1 0
2 0
3 0
4 0
5 0
